In [138]:
import os
if not os.path.exists('./eigen'):
    ! git clone https://gitlab.com/libeigen/eigen.git

In [139]:
import numba
from numba import jit
import time
import autograd.numpy as np

Original Functions:

In [140]:
def minibatch(data, batch_size):
    '''Define a function to compute the minibatches'''
    n = data.shape[0]
    p = data.shape[1]
    if n % batch_size != 0:
        n = (n // batch_size) * batch_size
    ind = np.arange(n)
    np.random.shuffle(ind)
    n_batches = n // batch_size
    data = data[ind].reshape(batch_size, p, n_batches)
    return(data, n_batches)

In [141]:
def sghmc(grad_U, eta, niter, alpha, theta_0, V_hat, dat, batch_size):
    '''Define SGHMC as described in the paper
    "Stochastic Gradient Hamiltonian Monte Carlo"
    by Tianqi Chen, Emily B. Fox, Carlos Guestrin 
    ICML (2014).
    
    The inputs include:
    grad_U = gradient of U
    eta = eps^2 M^(-1)
    niter = number of samples to generate
    alpha = eps M^(-1) C
    theta_0 = initial val of parameter(s) to be sampled
    V_hat = estimated covariance matrix of stoch grad noise
    The return is a np.array of positions of theta.'''

    p = len(theta_0) 
    n = dat.shape[0] 
    theta_samps = np.zeros((p, niter*(n // batch_size))) 
    beta_hat = 0.5 * V_hat @ eta
    Sigma = 2 * (alpha - beta_hat) @ eta
    if np.all(np.linalg.eigvals(Sigma)) <= 0: 
        print("Error: (alpha - beta_hat) eta is not positive definite")
        return
    if (batch_size > n): 
        print("Error: batch_size must be <= number of data points")
        return
    
    nu = np.random.multivariate_normal(np.zeros(p), eta).reshape(p,-1)
    theta = theta_0 
    it = 0
    for i in range(niter):
        dat_resh, nbatches = minibatch(dat, batch_size)
        nu = np.random.multivariate_normal(np.zeros(p), eta).reshape(p,-1)
        for batch in range(nbatches):
            grad_U_batch = grad_U(theta, dat_resh[:,:,batch], n, batch_size).reshape(p,-1)
            nu = nu - eta @ grad_U_batch - alpha @ nu + \
                 np.random.multivariate_normal(np.zeros(p), Sigma).reshape(p, -1)
            theta = theta + nu
            theta_samps[:,it] = theta.reshape(-1,p)
            it = it + 1
        
    return theta_samps

In [142]:
#simplified algorithm
def sghmc_simplified(grad_U, eta, niter, alpha, theta_0, V_hat, dat, batch_size):
    '''Define SGHMC as described in the paper
    Tianqi Chen, Emily B. Fox, Carlos Guestrin 
    Stochastic Gradient Hamiltonian Monte Carlo 
    ICML 2014.
    The inputs are:
    grad_U = gradient of U
    eta = eps^2 M^(-1)
    niter = number of samples to generate
    alpha = eps M^(-1) C
    theta_0 = initial val of parameter(s) to be sampled
    V_hat = estimated covariance matrix of stoch grad noise
    See paper for more details
    The return is a np.array of positions of theta.'''

    p = len(theta_0)
    n = dat.shape[0]
    theta_samps = np.zeros((p, niter*(n // batch_size)))
    beta_hat = 0.5 * V_hat @ eta
    Sigma = 2 * (alpha - beta_hat) @ eta
    Sig_chol = np.linalg.cholesky(Sigma)
    if np.all(np.linalg.eigvals(Sigma)) <= 0: 
        print("Error: (alpha - beta_hat) eta is not positive definite")
        return
    if (batch_size > n): 
        print("Error: batch_size must be <= number of data points")
        return

    nu = np.random.multivariate_normal(np.zeros(p), eta).reshape(p,-1)
    theta = theta_0
    eta_chol = np.linalg.cholesky(eta)
    it = 0
    for i in range(niter):
        dat_resh, nbatches = minibatch(dat, batch_size)
        nu = np.sqrt(eta_chol) @ np.random.normal(size=p).reshape(p,-1)
        for batch in range(nbatches):
            grad_U_batch = grad_U(theta, dat_resh[:,:,batch], n, batch_size).reshape(p,-1)
            nu = nu - eta @ grad_U_batch - alpha @ nu + \
                 Sig_chol @ np.random.normal(size=p).reshape(p,-1)
            theta = theta + nu
            theta_samps[:,it] = theta.reshape(-1,p)
            it = it + 1
        
    return theta_samps

Optimization, try to use JIT first

In [143]:
@jit
def minibatch_numba(data, batch_size):
    '''Define a function to compute the minibatches'''
    n = data.shape[0]
    p = data.shape[1]
    if n % batch_size != 0:
        n = (n // batch_size) * batch_size
    ind = np.arange(n)
    np.random.shuffle(ind)
    n_batches = n // batch_size
    data = data[ind].reshape(batch_size, p, n_batches)
    return(data, n_batches)

In [144]:
@jit
def sghmc_numba(grad_U, eta, niter, alpha, theta_0, V_hat, dat, batch_size):
    '''Define SGHMC as described in the paper
    "Stochastic Gradient Hamiltonian Monte Carlo"
    by Tianqi Chen, Emily B. Fox, Carlos Guestrin 
    ICML (2014).
    
    The inputs include:
    grad_U = gradient of U
    eta = eps^2 M^(-1)
    niter = number of samples to generate
    alpha = eps M^(-1) C
    theta_0 = initial val of parameter(s) to be sampled
    V_hat = estimated covariance matrix of stoch grad noise
    The return is a np.array of positions of theta.'''

    p = len(theta_0)
    n = dat.shape[0]
    theta_samps = np.zeros((p, niter*(n // batch_size)))
    beta_hat = 0.5 * V_hat @ eta
    Sigma = 2 * (alpha - beta_hat) @ eta
    Sig_chol = np.linalg.cholesky(Sigma)
    if np.all(np.linalg.eigvals(Sigma)) <= 0: 
        print("Error: (alpha - beta_hat) eta not pos def")
        return
    if (batch_size > n): 
        print("Error: batch_size must be <= number of data points")
        return
    nu = np.random.multivariate_normal(np.zeros(p), eta).reshape(p,-1)
    theta = theta_0
    eta_chol = np.linalg.cholesky(eta)
    it = 0
    for i in range(niter):
        dat_resh, nbatches = minibatch_numba(dat, batch_size)
        nu = eta_chol @ np.random.normal(size=p).reshape(p,-1) 
        for batch in range(nbatches):
            gradU_batch = grad_U(theta, dat_resh[:,:,batch], n, batch_size).reshape(p,-1)
            nu = nu - eta @ gradU_batch - alpha @ nu + \
                 Sig_chol @ np.random.normal(size=p).reshape(p,-1)
            theta = theta + nu
            theta_samps[:,it] = theta.reshape(-1,p)
            it = it + 1
        
    return theta_samps

In [145]:
#comparsion for mvn

from autograd import jacobian

def log_prior(theta):
    return(-(1/(2*10))*theta.T@theta)
      
def log_lik(theta, x):
    return(np.log(0.5 * np.exp(-0.5*(theta[0]-x)**2) + 0.5* np.exp(-0.5*(theta[1]-x)**2)))

def U(theta, x, n, batch_size):
    return(-log_prior(theta) - (n/batch_size)*sum(log_lik(theta, x)))
       
gradU = jacobian(U, argnum=0)

np.random.seed(1234)
p = 2
theta = np.array([-3.0, 3.0]).reshape(p,-1)
n = 200
x = np.array([np.random.normal(theta[0], 1, (n,1)),
              np.random.normal(theta[1], 1, (n,1))]).reshape(-1,1)
theta_0 = theta
eta = 0.01/n * np.eye(p)
alpha = 0.1 * np.eye(p)
V = np.eye(p)*1
niter = 500
batch_size=50

In [146]:
%%file project.cpp
<%
cfg['compiler_args'] = ['-std=c++11']
cfg['include_dirs'] = ['eigen']
setup_pybind11(cfg)
%>

#include <pybind11/pybind11.h>
#include <pybind11/eigen.h>
#include <stdexcept>
#include <algorithm> // std::random_shuffle
#include <random>

#include <Eigen/LU>
#include <Eigen/Dense>
#include <eigen/Eigen/Core>

namespace py = pybind11;
using std::default_random_engine;
using std::normal_distribution;
        
default_random_engine re{100};
normal_distribution<double> norm(0, 1);
auto rnorm = bind(norm, re);

Eigen::MatrixXd rnorm_vec(int n) {
    Eigen::MatrixXd res_vec = Eigen::MatrixXd::Zero(n, 1);
    for (int i=0; i<n; i++) {
        res_vec(i,0) = rnorm();
    }
    return res_vec;
}
    
Eigen::MatrixXd gradU_noisyFig1(Eigen::MatrixXd theta) {
    Eigen::MatrixXd xs = -4*theta.array() + 4*theta.array().pow(3) + 2*rnorm_vec(theta.rows()).array();
    return xs;
}
     
Eigen::MatrixXd gradU_mixNormals(Eigen::MatrixXd theta, Eigen::MatrixXd x, int n, int batch_size) {
    int p = theta.rows();
    Eigen::ArrayXd c_0 = theta(0,0) - x.array();
    Eigen::ArrayXd c_1 = theta(1,0) - x.array();
    Eigen::ArrayXd star = 0.5 * (-0.5 * c_0.pow(2)).exp() + 0.5 * (-0.5 * c_1.pow(2)).exp();
    Eigen::ArrayXd star_prime;
    Eigen::MatrixXd grad = Eigen::MatrixXd::Zero(p, 1);
    for (int i=0; i<p; i++) {
        star_prime = 0.5 * (-0.5 * (theta(i,0) - x.array()).pow(2)).exp() * (theta(i,0) - x.array());
        grad(i,0) = -theta(i,0)/10 - (n/batch_size)*(star_prime/star).sum();
    }
    return grad;
} 

Eigen::MatrixXd sghmc(std::string gradU_choice, Eigen::MatrixXd eta, int niter, Eigen::MatrixXd alpha, Eigen::MatrixXd theta_0, Eigen::MatrixXd V_hat, Eigen::MatrixXd dat, int batch_size){
    int p = theta_0.rows();
    int n = dat.rows();     
    int p_dat = dat.cols();
    int nbatches = n / batch_size; 
    Eigen::MatrixXd dat_temp = dat;
    Eigen::MatrixXd dat_batch = Eigen::MatrixXd::Zero(batch_size, p_dat);
    Eigen::MatrixXd gradU_batch = Eigen::MatrixXd::Zero(p, 1);
    Eigen::MatrixXd theta_samps = Eigen::MatrixXd::Zero(p, niter*(n/batch_size));
    std::vector<int> ind;
    Eigen::MatrixXd beta_hat = 0.5 * V_hat * eta;
    Eigen::MatrixXd Sigma = 2.0 * (alpha - beta_hat) * eta;
    Eigen::LLT<Eigen::MatrixXd> lltOfSig(Sigma); 
    if(lltOfSig.info() == Eigen::NumericalIssue){ 
        return theta_samps; 
    }
    Eigen::MatrixXd Sig_chol = lltOfSig.matrixL(); 
    if(batch_size > n){ 
        return theta_samps; 
    }
    Eigen::LLT<Eigen::MatrixXd> lltOfeta(eta); 
    Eigen::MatrixXd eta_chol = lltOfeta.matrixL(); 
    Eigen::MatrixXd nu = eta_chol * rnorm_vec(p); 
    Eigen::MatrixXd theta = theta_0; 
    
    int big_iter = 0;
    for (int it=0; it<niter; it++) {
        
        Eigen::VectorXi indices = Eigen::VectorXi::LinSpaced(dat.rows(), 0, dat.rows());
        std::random_shuffle(indices.data(), indices.data() + dat.rows());
        dat_temp = indices.asPermutation() * dat;
        nu = eta_chol * rnorm_vec(p);
        int count_lower = 0;
        int count_upper = batch_size;
        for (int b=0; b<nbatches; b++){
            int batch_ind = 0;
            for (int ind_temp=count_lower; ind_temp<count_upper; ind_temp++){
                dat_batch.row(batch_ind) = dat_temp.row(ind_temp);
                batch_ind += 1;
            }
            count_lower += batch_size;
            count_upper += batch_size;
            if (gradU_choice == "fig1"){
                gradU_batch = gradU_noisyFig1(theta);
            } else if (gradU_choice == "mixture_of_normals"){
                gradU_batch = gradU_mixNormals(theta, dat_batch, n, batch_size);
            } else {
                return theta_samps;
            }
            nu = nu - eta * gradU_batch - alpha * nu + Sig_chol * rnorm_vec(p);
            theta = theta + nu;
            theta_samps.col(big_iter) = theta;
            big_iter += 1;
        }
    }

    return theta_samps;
}
    
PYBIND11_MODULE(project, m) {
    m.doc() = "auto-compiled c++ extension";
    m.def("sghmc", &sghmc);
}

Overwriting project.cpp


In [147]:
%timeit sghmc(gradU, eta, niter, alpha, theta_0, V, x, batch_size)

23 s ± 104 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [148]:
%timeit sghmc_simplified(gradU, eta, niter, alpha, theta_0, V, x, batch_size)

22.2 s ± 90.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [149]:
%timeit sghmc_numba(gradU, eta, niter, alpha, theta_0, V, x, batch_size)

<ipython-input-144-02c145d81987>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "sghmc_numba" failed type inference due to: non-precise type pyobject
[1] During: typing of argument at <ipython-input-144-02c145d81987> (17)

File "<ipython-input-144-02c145d81987>", line 17:
def sghmc_numba(grad_U, eta, niter, alpha, theta_0, V_hat, dat, batch_size):
    <source elided>

    p = len(theta_0)
    ^

  @jit
<ipython-input-144-02c145d81987>:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "sghmc_numba" failed type inference due to: cannot determine Numba type of <class 'numba.dispatcher.LiftedLoop'>

File "<ipython-input-144-02c145d81987>", line 33:
def sghmc_numba(grad_U, eta, niter, alpha, theta_0, V_hat, dat, batch_size):
    <source elided>
    it = 0
    for i in range(niter):
    ^

  @jit
/opt/conda/lib/python3.6/site-packages/numba/object_mode_passes.py:178: NumbaWarning: 

21.7 s ± 180 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [150]:
%%file test1.py
import cppimport
import numpy as np
import seaborn as sns
import time

test = cppimport.imp("project")

if __name__ == '__main__':
    np.random.seed(1234)
    np.random.seed(1234)
    p = 2 
    theta = np.array([-3.0, 3.0]).reshape(p,-1)
    n = 200 
    x = np.array([np.random.normal(theta[0], 1, (n,1)),
                  np.random.normal(theta[1], 1, (n,1))]).reshape(-1,1)
    theta_0 = theta
    eta = 0.01/n * np.eye(p)
    alpha = 0.1 * np.eye(p)
    V = np.eye(p)*1
    niter = 500
    batch_size=50
    times_all = np.zeros(7)
    for i in range(7):
        t0 = time.time()
        samps_sghmc = test.sghmc("mixture_of_normals", eta, niter, alpha, theta_0, V, x, batch_size)
        t1 = time.time()
        times_all[i] = t1 - t0
    print(times_all.mean(),"s ±",times_all.std(), 
          "s per loop (mean ± std. dev. of 7 runs, 1 loop each)")

Overwriting test1.py


In [151]:
%%bash

python test1.py

0.019376618521554128 s ± 0.0003748492705152947 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [152]:
#comparsion for fig 1

import time
import autograd.numpy as np
from autograd import jacobian

def U(theta):
    return(-2*theta**2 + theta**4)
gradU = jacobian(U, argnum=0)
def noisy_gradU(theta, x, n, batch_size):
    return -4*theta + 4*theta**3 + np.random.normal(0,2)

np.random.seed(1234)
n = 100
x = np.array([np.random.normal(0, 1, (n,1))]).reshape(-1,1)
theta_0 = np.array([0.0])
p = theta_0.shape[0]
eta = 0.001 * np.eye(p) 
alpha = 0.01 * np.eye(p)
V = np.eye(p)
batch_size = n
niter = 50000

In [153]:
times_all = np.zeros(7)
for i in range(7):
    #original 
    t0 = time.time()
    samps_sghmc = sghmc(noisy_gradU, eta, niter, alpha, theta_0, V, x, batch_size)
    t1 = time.time()
    times_all[i] = t1 - t0
print(times_all.mean(),"s ±",times_all.std(), 
      "s per loop (mean ± std. dev. of 7 runs, 1 loop each)")

18.179001126970565 s ± 0.04150309486923006 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [154]:
times_all = np.zeros(7)
for i in range(7):
    #simplified
    t0 = time.time()
    samps_sghmc = sghmc_simplified(noisy_gradU, eta, niter, alpha, theta_0, V, x, batch_size)
    t1 = time.time()
    times_all[i] = t1 - t0
print(times_all.mean(),"s ±",times_all.std(), 
      "s per loop (mean ± std. dev. of 7 runs, 1 loop each)")

3.2300535270145962 s ± 0.01802794287233938 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [155]:
times_all = np.zeros(7)
for i in range(7):
    #numba
    t0 = time.time()
    samps_sghmc = sghmc_numba(noisy_gradU, eta, niter, alpha, theta_0, V, x, batch_size)
    t1 = time.time()
    times_all[i] = t1 - t0
print(times_all.mean(),"s ±",times_all.std(), 
      "s per loop (mean ± std. dev. of 7 runs, 1 loop each)")

<ipython-input-144-02c145d81987>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "sghmc_numba" failed type inference due to: non-precise type pyobject
[1] During: typing of argument at <ipython-input-144-02c145d81987> (17)

File "<ipython-input-144-02c145d81987>", line 17:
def sghmc_numba(grad_U, eta, niter, alpha, theta_0, V_hat, dat, batch_size):
    <source elided>

    p = len(theta_0)
    ^

  @jit
<ipython-input-144-02c145d81987>:1: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled because Function "sghmc_numba" failed type inference due to: cannot determine Numba type of <class 'numba.dispatcher.LiftedLoop'>

File "<ipython-input-144-02c145d81987>", line 33:
def sghmc_numba(grad_U, eta, niter, alpha, theta_0, V_hat, dat, batch_size):
    <source elided>
    it = 0
    for i in range(niter):
    ^

  @jit
/opt/conda/lib/python3.6/site-packages/numba/object_mode_passes.py:178: NumbaWarning: 

4.451998233795166 s ± 0.8858564442586407 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [156]:
%%file test2.py
import cppimport
import numpy as np
import seaborn as sns
import time

test = cppimport.imp("project")

if __name__ == '__main__':
    np.random.seed(1234)
    n = 100
    x = np.array([np.random.normal(0, 1, (n,1))]).reshape(-1,1)
    theta_0 = np.array([0.0]) 
    p = theta_0.shape[0]
    eta = 0.001 * np.eye(p) 
    alpha = 0.01 * np.eye(p)
    V = np.eye(p)
    batch_size = n
    niter = 50000 
    
    times_all = np.zeros(7)
    for i in range(7):
        t0 = time.time()
        samps_sghmc = test.sghmc("fig1", eta, niter, alpha, theta_0, V, x, batch_size)
        t1 = time.time()
        times_all[i] = t1 - t0
    print(times_all.mean(),"s ±",times_all.std(), 
          "s per loop (mean ± std. dev. of 7 runs, 1 loop each)")

Overwriting test2.py


In [157]:
%%bash
python test2.py

0.20663997105189733 s ± 0.0038836895140119477 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
